In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.7 MB/s eta 0:00:00


In [ ]:
!pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=77b182da17fb5a941fea96ba1c631d39ab0285cd65ce0e84433eb95af6a8ead4
  Stored in directory: /root/.cache/pip/wheels/c2/46/f4/caa1bee71096d7b0cdca2f2a2af45cacf35c5760bee8f00948
Successfully built wikipedia


# Import modules

In [ ]:
import wikipedia
import torch
from transformers import AutoTokenizer, AutoModel
from scipy.spatial import distance

# Load the pretrained model

In [ ]:
# load pre-trained BERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# set device to CUDA if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

# Bert Embeddings

In [ ]:
# Returns the BERT embeddings for the given text
def get_bert_embeddings(text):
    
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True, max_length = 512)).unsqueeze(0)
    input_ids = input_ids.to(device)
    outputs = model(input_ids)
    last_hidden_state = outputs.last_hidden_state
    embeddings = torch.mean(last_hidden_state, dim=1).squeeze()
    return embeddings.detach().cpu().numpy()

# Search Method

In [ ]:
# Returns the top 'n_results' search results from Wikipedia for the given query
def search_wikipedia(query, n_results=5, similarity_threshold = 0.5 ):
    
    # get search results from Wikipedia API
    search_results = wikipedia.search(query, results=n_results)

    # initialize list to store results
    results = []

    # iterate over search results and get BERT embeddings for each page summary
    try:
      for result in search_results:
          try:
              # get page summary
              page = wikipedia.page(result)
              summary = page.summary

              # get BERT embeddings for query and page summary
              query_embeddings = get_bert_embeddings(query)
              summary_embeddings = get_bert_embeddings(summary)

              # calculate cosine similarity between query and page summary embeddings
              similarity = 1 - (distance.cosine(query_embeddings, summary_embeddings))
        
              # filter out results with similarity score below threshold
              if similarity < similarity_threshold:
                  continue
              # add result to list
              results.append({'title': page.title, 'url': page.url, 'summary': summary, 'similarity': similarity})
          except wikipedia.exceptions.DisambiguationError as e:
              # if page is disambiguation page, skip it
              continue
    except:
      print("Page Not Found")

    # sort results by similarity in descending order
    results = sorted(results, key=lambda x: x['similarity'], reverse=True)

    # return top n_results results
    return results[:n_results]


Query insertion and result generation

In [ ]:
# get user input
query = input("Enter your query: ")

# search wikipedia for query
results = search_wikipedia(query)

# print search results
for i, result in enumerate(results):
    print(f"\nResult {i+1}:")
    print(f"Title: {result['title']}")
    print(f"URL: {result['url']}")
    print(f"Summary: {result['summary']}")

Enter your query: Indraprastha Institute of Information Technology


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Page Not Found

Result 1:
Title: Indraprastha Institute of Information Technology, Delhi
URL: https://en.wikipedia.org/wiki/Indraprastha_Institute_of_Information_Technology,_Delhi
Summary: Indraprastha Institute of Information Technology, Delhi (Iṃdraprastha Sūcanā Praudyōgikī Saṃsthān Dillī, IIIT-Delhi or IIIT-D) is an autonomous State University located in Delhi, India. It is a research-oriented institute with a focus on Computer Science and allied areas. IIIT delhi offers B.Tech,M.Tech and Ph.D degrees.

Result 2:
Title: Institute of Information Technology and Management
URL: https://en.wikipedia.org/wiki/Institute_of_Information_Technology_and_Management
Summary: Institute of Information Technology & Management is a self-financed institute affiliated to Guru Gobind Singh Indraprastha University and located in Janakpuri, New Delhi. IITM is recognised as non-government unaided Institute under section 2(f) by UGC.

Result 3:
Title: Guru Gobind Singh Indraprastha University
URL: https:

# Precision, Recall and F1-score for the given query 

In [ ]:
import pandas as pd

# create a test dataset with query and relevant page titles
test_data = {'query': ['Narendra Modi','Indraprastha Institute of Information Technology'],
             'relevant_pages': [['Narendra Modi', 'PM Narendra Modi'],['Indraprastha Institute of Information Technology, Delhi','Higher Education']]}
test_df = pd.DataFrame(test_data)

# evaluate search engine on test dataset
def evaluate_search_engine(test_df, n_results=5):
    results = []
    for index, row in test_df.iterrows():
        query = row['query']
        relevant_pages = row['relevant_pages']
        search_results = search_wikipedia(query, n_results=n_results)
        retrieved_pages = [result['title'] for result in search_results]
        precision = len(set(retrieved_pages).intersection(set(relevant_pages))) / n_results
        print(precision)
        recall = len(set(retrieved_pages).intersection(set(relevant_pages))) / len(relevant_pages)
        print(recall)
        f1_score = 2 * precision * recall / (precision + recall)
        results.append({'query': query, 'precision': precision, 'recall': recall, 'f1_score': f1_score})
    return pd.DataFrame(results)

# evaluate search engine on test dataset
evaluation_results = evaluate_search_engine(test_df)

# print evaluation results
print(evaluation_results)


0.4
1.0
Page Not Found
0.2
0.5
                                              query  precision  recall  \
0                                     Narendra Modi        0.4     1.0   
1  Indraprastha Institute of Information Technology        0.2     0.5   

   f1_score  
0  0.571429  
1  0.285714  


# Accuracy Score

In [ ]:
# define a set of test queries and expected results
test_queries = ["Narendra Modi",'Indraprastha Institute of Information Technology']
expected_results = [
    ["Narendra Modi", "PM Narendra Modi"],['Indraprastha Institute of Information Technology, Delhi','Higher Education']
]

# search wikipedia for each test query and compare results to expected results
for i, query in enumerate(test_queries):
    print(f"Test query {i+1}: {query}")
    
    # get search results from wikipedia search engine
    search_results = search_wikipedia(query)
    
    # extract titles from search results
    search_titles = [result["title"] for result in search_results]
    
    # compare search titles to expected titles
    expected_titles = expected_results[i]
    num_correct = len(set(search_titles) & set(expected_titles))
    num_total = len(expected_titles)
    accuracy = num_correct / num_total
    
    # print evaluation results
    print(f"Expected titles: {expected_titles}")
    print(f"Search titles: {search_titles}")
    print(f"Accuracy: {accuracy}\n")

Test query 1: Narendra Modi
Expected titles: ['Narendra Modi', 'PM Narendra Modi']
Search titles: ['PM Narendra Modi', 'Jashodaben Modi', 'Narendra Modi', 'Narendra Modi Stadium', 'Premiership of Narendra Modi']
Accuracy: 1.0

Test query 2: Indraprastha Institute of Information Technology
Page Not Found
Expected titles: ['Indraprastha Institute of Information Technology, Delhi', 'Higher Education']
Search titles: ['Indraprastha Institute of Information Technology, Delhi', 'Institute of Information Technology and Management', 'Guru Gobind Singh Indraprastha University', 'Kiran Karnik']
Accuracy: 0.5



# Mean reciprocal rank

In [ ]:
# define a set of test queries and expected results
test_queries = ["Narendra Modi",'Indraprastha Institute of Information Technology']
expected_results = [
    ["Narendra Modi", "PM Narendra Modi"],['Indraprastha Institute of Information Technology, Delhi','Higher Education']
]

# calculate MRR for each test query
mrr_sum = 0.0
for i, query in enumerate(test_queries):
    print(f"Test query {i+1}: {query}")
    
    # get search results from wikipedia search engine
    search_results = search_wikipedia(query)
    
    # extract titles from search results
    search_titles = [result["title"] for result in search_results]
    
    # calculate MRR for this query
    expected_titles = expected_results[i]
    mrr = 0.0
    for j, title in enumerate(search_titles):
        if title in expected_titles:
            rank = j + 1
            mrr = 1.0 / rank
            break
    
    # add MRR for this query to sum
    mrr_sum += mrr
    
    # print evaluation results
    print(f"Expected titles: {expected_titles}")
    print(f"Search titles: {search_titles}")
    print(f"MRR: {mrr}\n")

# calculate average MRR across all test queries
num_queries = len(test_queries)
avg_mrr = mrr_sum / num_queries
print(f"Average MRR: {avg_mrr}")


Test query 1: Narendra Modi
Expected titles: ['Narendra Modi', 'PM Narendra Modi']
Search titles: ['PM Narendra Modi', 'Jashodaben Modi', 'Narendra Modi', 'Narendra Modi Stadium', 'Premiership of Narendra Modi']
MRR: 1.0

Test query 2: Indraprastha Institute of Information Technology
Page Not Found
Expected titles: ['Indraprastha Institute of Information Technology, Delhi', 'Higher Education']
Search titles: ['Indraprastha Institute of Information Technology, Delhi', 'Institute of Information Technology and Management', 'Guru Gobind Singh Indraprastha University', 'Kiran Karnik']
MRR: 1.0

Average MRR: 1.0


# Mean Average Precision

In [ ]:
# define a set of test queries and expected results
test_queries = ["Narendra Modi",'Indraprastha Institute of Information Technology']
expected_results = [
    ["Narendra Modi", "PM Narendra Modi"],['Indraprastha Institute of Information Technology, Delhi','Higher Education']
]

# calculate MAP for each test query
map_sum = 0.0
for i, query in enumerate(test_queries):
    print(f"Test query {i+1}: {query}")
    
    # get search results from wikipedia search engine
    search_results = search_wikipedia(query)
    
    # extract titles from search results
    search_titles = [result["title"] for result in search_results]
    
    # calculate MAP for this query
    expected_titles = expected_results[i]
    num_correct = 0.0
    precision_sum = 0.0
    for j, title in enumerate(search_titles):
        if title in expected_titles:
            num_correct += 1.0
            precision = num_correct / (j+1)
            precision_sum += precision
    
    # calculate average precision for this query
    avg_precision = precision_sum / len(expected_titles)
    
    # add average precision for this query to sum
    map_sum += avg_precision
    
    # print evaluation results
    print(f"Expected titles: {expected_titles}")
    print(f"Search titles: {search_titles}")
    print(f"Average precision: {avg_precision}\n")

# calculate average MAP across all test queries
num_queries = len(test_queries)
avg_map = map_sum / num_queries
print(f"Average MAP: {avg_map}")


Test query 1: Narendra Modi
Expected titles: ['Narendra Modi', 'PM Narendra Modi']
Search titles: ['PM Narendra Modi', 'Jashodaben Modi', 'Narendra Modi', 'Narendra Modi Stadium', 'Premiership of Narendra Modi']
Average precision: 0.8333333333333333

Test query 2: Indraprastha Institute of Information Technology
Page Not Found
Expected titles: ['Indraprastha Institute of Information Technology, Delhi', 'Higher Education']
Search titles: ['Indraprastha Institute of Information Technology, Delhi', 'Institute of Information Technology and Management', 'Guru Gobind Singh Indraprastha University', 'Kiran Karnik']
Average precision: 0.5

Average MAP: 0.6666666666666666


# User Feedback

In [ ]:
def get_user_feedback(results):
    feedback = []
    for result in results:
        relevance = input(f"Is the result '{result['title']}' relevant? (y/n): ")
        if relevance == 'y':
            feedback.append(1)
        else:
            feedback.append(0)
    return feedback

In [ ]:
def refine_search_query(query, results, feedback):
    new_query = query
    for i, result in enumerate(results):
        if feedback[i] == 0:
            new_query += f" -\"{result['title']}\""
    return new_query

In [ ]:
# ask the user for feedback and refine the search query
feedback = get_user_feedback(results)
new_query = refine_search_query(query, results, feedback)
results = search_wikipedia(new_query)
# print search results
for i, result in enumerate(results):
    print(f"\nResult {i+1}:")
    print(f"Title: {result['title']}")
    print(f"URL: {result['url']}")
    print(f"Summary: {result['summary']}")

# ask the user for feedback and refine the search query again (repeat as needed)
feedback = get_user_feedback(results)
new_query = refine_search_query(new_query, results, feedback)
results = search_wikipedia(new_query)
# print search results
for i, result in enumerate(results):
    print(f"\nResult {i+1}:")
    print(f"Title: {result['title']}")
    print(f"URL: {result['url']}")
    print(f"Summary: {result['summary']}")
# etc.

Is the result 'Indraprastha Institute of Information Technology, Delhi' relevant? (y/n): y
Is the result 'Institute of Information Technology and Management' relevant? (y/n): n
Is the result 'Guru Gobind Singh Indraprastha University' relevant? (y/n): n
Is the result 'Kiran Karnik' relevant? (y/n): n

Result 1:
Title: Pankaj Jalote
URL: https://en.wikipedia.org/wiki/Pankaj_Jalote
Summary: Pankaj Jalote was the Director of Indraprastha Institute of Information Technology. 
Jalote is a Fellow of the IEEE and INAE.Before joining IIIT Delhi, he worked as the Microsoft Chair Professor at the Department of Computer Science and Engineering at IIT Delhi. Jalote has also taught at the Department of Computer Science at IIT Kanpur and University of Maryland.

Result 2:
Title: Gajendra Pal Singh Raghava
URL: https://en.wikipedia.org/wiki/Gajendra_Pal_Singh_Raghava
Summary: Gajendra Pal Singh Raghava is an Indian bio-informatician and head of computational biology at the Indraprastha Institute of I